<a href="https://colab.research.google.com/github/yuann403/financial/blob/main/week13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

使用基因演算法實現最佳投資組合選擇。
* 每個染色體代表一個投資組合（由多支股票組成，基因為0或1，表示是否選擇該股票）。
* 可根據自定義的適應函數（如回報率、風險等）評估投資組合的優劣，並通過初始化族群、選擇、交配及突變，優化出最佳投資組合，最後輸出其回報率、風險和選股結果。

In [1]:
import yfinance as yf
import numpy as np

In [2]:
# 定義目標台股股票池
tickers = ['2330.TW', '2881.TW', '2454.TW']
risk_free_rate = 0.01  # 無風險利率，假設為1%

# 下載股票歷史數據
data = yf.download(tickers, start="2020-01-01", end="2023-01-01", interval="1d")['Adj Close']

# 計算每日回報率
returns = data.pct_change().dropna()

# 計算年化平均回報率和協方差矩陣
annual_returns = returns.mean() * 252  # 假設一年252個交易日
cov_matrix = returns.cov() * 252

[*********************100%***********************]  3 of 3 completed


In [3]:
# 適應函數
def evaluate_portfolio(chromosome):
    selected_indices = np.where(chromosome == 1)[0]
    if len(selected_indices) == 0:  # 避免空投資組合
        return 0.01  # 返回一個非常小的值以避免無法計算
    portfolio_return = np.sum(annual_returns.iloc[selected_indices]) / len(selected_indices)
    portfolio_risk = np.sqrt(np.dot(chromosome.T, np.dot(cov_matrix.to_numpy(), chromosome)))
    if portfolio_risk == 0:  # 避免風險為零的情況
        return 0.01
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_risk
    return max(sharpe_ratio, 0.01)  # 確保適應函數值為正

# 初始化種群
def initialize_population(population_size, num_stocks):
    return np.random.randint(0, 2, (population_size, num_stocks))

# 選擇（輪盤選擇法）
def select_population(population, fitness_scores):
    fitness_scores = np.maximum(fitness_scores, 0.01)  # 確保適應度值為正
    probabilities = fitness_scores / np.sum(fitness_scores)
    indices = np.random.choice(range(len(population)), size=len(population), p=probabilities)
    return population[indices]

# 交叉（單點交叉）
def crossover(parent1, parent2):
    point = np.random.randint(1, len(parent1) - 1)
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    return child1, child2

# 變異
def mutate(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if np.random.rand() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

In [4]:
# 基因演算法參數
population_size = 50
generations = 100
mutation_rate = 0.02
num_stocks = len(tickers)

# 基因演算法主程序
population = initialize_population(population_size, num_stocks)

for generation in range(generations):
    # 計算適應度
    fitness_scores = np.array([evaluate_portfolio(chromosome) for chromosome in population])

    # 選擇新種群
    population = select_population(population, fitness_scores)

    # 交叉與變異
    new_population = []
    for i in range(0, len(population), 2):
        parent1, parent2 = population[i], population[min(i + 1, len(population) - 1)]
        child1, child2 = crossover(parent1, parent2)
        new_population.extend([mutate(child1, mutation_rate), mutate(child2, mutation_rate)])
    population = np.array(new_population)

    # 印出每代的最佳結果
    best_index = np.argmax(fitness_scores)
    best_portfolio = population[best_index]
    best_return = np.sum(annual_returns[best_portfolio == 1])
    best_risk = np.sqrt(np.dot(best_portfolio.T, np.dot(cov_matrix.to_numpy(), best_portfolio)))
    print(f"Generation {generation + 1}: Best Return={best_return:.4f}, Best Risk={best_risk:.4f}")

Generation 1: Best Return=0.3609, Best Risk=0.4665
Generation 2: Best Return=0.6185, Best Risk=0.7741
Generation 3: Best Return=0.2575, Best Risk=0.4144
Generation 4: Best Return=0.2000, Best Risk=0.2557
Generation 5: Best Return=0.0000, Best Risk=0.0000
Generation 6: Best Return=0.2000, Best Risk=0.2557
Generation 7: Best Return=0.2000, Best Risk=0.2557
Generation 8: Best Return=0.2000, Best Risk=0.2557
Generation 9: Best Return=0.2000, Best Risk=0.2557
Generation 10: Best Return=0.2000, Best Risk=0.2557
Generation 11: Best Return=0.2000, Best Risk=0.2557
Generation 12: Best Return=0.2000, Best Risk=0.2557
Generation 13: Best Return=0.2575, Best Risk=0.4144
Generation 14: Best Return=0.2000, Best Risk=0.2557
Generation 15: Best Return=0.2000, Best Risk=0.2557
Generation 16: Best Return=0.2000, Best Risk=0.2557
Generation 17: Best Return=0.2000, Best Risk=0.2557
Generation 18: Best Return=0.2000, Best Risk=0.2557
Generation 19: Best Return=0.2000, Best Risk=0.2557
Generation 20: Best R

In [5]:
# 最終結果
best_index = np.argmax(fitness_scores)
best_portfolio = population[best_index]
selected_stocks = [tickers[i] for i in range(num_stocks) if best_portfolio[i] == 1]
best_return = np.sum(annual_returns[best_portfolio == 1])
best_risk = np.sqrt(np.dot(best_portfolio.T, np.dot(cov_matrix.to_numpy(), best_portfolio)))

print("最佳投資組合：")
print(f"選擇的股票：{selected_stocks}")
print(f"投資組合的回報率：{best_return:.4f}")
print(f"投資組合的風險：{best_risk:.4f}")

最佳投資組合：
選擇的股票：['2454.TW']
投資組合的回報率：0.2000
投資組合的風險：0.2557
